In [1]:
# !pip install psycopg2
# !pip install google-cloud-sql

In [2]:
import pandas as pd
import numpy as np
import pickle
import json
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import psycopg2

In [3]:
# #@markdown Please fill in the value below with your GCP project ID and then run the cell.

# # Please fill in these values.
# project_id = "digital-display-376201" #@param {type:"string"}

# # Quick input validations.
# assert project_id, "⚠️ Please provide a Google Cloud project ID"

# # Configure gcloud.
# !gcloud config set project {project_id}

In [4]:
# from google.cloud.sql.connector import Connector
# import sqlalchemy

In [5]:
# preparamos o driver do Chrome
driver = webdriver.Chrome('path/chromedriver')

<ipython-input-5-c16ba3ec765d>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('path/chromedriver')


In [6]:
!dir data_output

 Volume in drive C is OS
 Volume Serial Number is 2CDA-BFF4

 Directory of C:\Users\Ernesto\OneDrive - usp.br\Turing USP\DS\data_engineering\crawler\data_output

02/28/2023  12:57 AM    <DIR>          .
02/28/2023  12:57 AM    <DIR>          ..
12/11/2022  09:56 AM    <DIR>          .ipynb_checkpoints
12/04/2022  05:01 PM         1,446,871 1027_2000.csv
12/04/2022  10:40 PM         2,172,837 2000_5000.csv
12/04/2022  03:08 PM           743,293 first_1027.csv
02/27/2023  04:31 AM        78,682,300 movie_ids_02_27_2023.json
01/29/2023  07:56 PM               228 Rotten_2021_4000.csv
12/09/2022  03:06 AM            90,278 rotten_tomatoes_with_titles.ipynb
12/11/2022  12:08 PM           842,443 TMDB_2021_4000.csv
02/28/2023  12:57 AM             7,372 ztrending_movies.csv
               8 File(s)     83,985,622 bytes
               3 Dir(s)  14,920,404,992 bytes free


In [7]:
file = 'data_output/ztrending_movies.csv'

df0 = pd.read_csv(file).head(6)

In [8]:
df0

,id,title,original_title,overview,poster_path,media_type,popularity,release_date,vote_average,vote_count,imdb_id,status,revenue,budget,collection,productions,genres,release_year,type
0,673,Harry Potter e o Prisioneiro de Azkaban,Harry Potter and the Prisoner of Azkaban,Quando a desprezível Tia Guida voa pelo céu na...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,227.765,2004-05-31,8.021,19139,tt0304141,Released,789804554,130000000,Harry Potter: Coleção,"['Warner Bros. Pictures', '1492 Pictures', 'He...","['Aventura', 'Fantasia']",2004,trending_week
1,299537,Capitã Marvel,Captain Marvel,"Capitã Marvel, parte do exército de elite dos ...",https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,85.375,2019-03-06,6.872,13984,tt4154664,Released,1131416446,152000000,Capitã Marvel: Coleção,['Marvel Studios'],"['Ação', 'Aventura', 'Ficção científica']",2019,trending_week
2,335984,Blade Runner 2049,Blade Runner 2049,Após descobrir um segredo enterrado há muito t...,https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,117.601,2017-10-04,7.523,11591,tt1856101,Released,259239658,150000000,Blade Runner: Coleção,"['Columbia Pictures', 'Warner Bros. Pictures',...","['Ficção científica', 'Drama']",2017,trending_week
3,1771,Capitão América: O Primeiro Vingador,Captain America: The First Avenger,"Durante a Segunda Guerra Mundial, Steve Rogers...",https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,69.620,2011-07-22,6.988,19572,tt0458339,Released,370569774,140000000,Capitão América: Coleção,['Marvel Studios'],"['Ação', 'Aventura', 'Ficção científica']",2011,trending_week
4,2034,Dia de Treinamento,Training Day,"Jake Hoyt, um jovem policial de Los Angeles, e...",https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,62.174,2001-10-05,7.466,5053,tt0139654,Released,104876233,45000000,Training Day Collection,"['Village Roadshow Pictures', 'NPV Entertainme...","['Ação', 'Crime', 'Drama']",2001,trending_week
5,121,O Senhor dos Anéis: As Duas Torres,The Lord of the Rings: The Two Towers,"Após a captura de Merry e Pippy pelos orcs, a ...",https://image.tmdb.org/t/p/w220_and_h330_face/...,movie,103.034,2002-12-18,8.375,19391,tt0167261,Released,926287400,79000000,O Senhor dos Anéis: Coleção,"['WingNut Films', 'New Line Cinema', 'The Saul...","['Aventura', 'Fantasia', 'Ação']",2002,trending_week


In [9]:
df0.id

0       673
1    299537
2    335984
3      1771
4      2034
5       121
Name: id, dtype: int64

# Criacao do dataset com os dados do Rotten Tomatoes

In [10]:
# criamos esta funcao para que os nomes dos filmes sejam transformados nos nomes dos filmes da url
# exemplo: The Matrix ---> the_matrix, desta forma conseguimos acessar as urls

def rottenize(title):
    '''
    Esta funcao sustitui os carateres por vazio ou por _
    para depois entrar nos links (urls) dos filmes no rotten
    '''
    title = title.translate(str.maketrans({':': '', "'": '', ".":"", ",":"", 
                                          "-":"_", " ":"_"}))
    
    return title.lower()

In [11]:
# criamos uma lista vazia para armazenar as urls dos filmes do dataset do TMDB
links = []
for tit in df0['original_title'].apply(rottenize):
    links.append('https://www.rottentomatoes.com/m/{}'.format(tit))

In [12]:
# listas para armazenar a informacao
tmdb_id = []
nomes = []
TOMATOMETER = []
TOMATOMETER_rev = []
aud_scr = []
aud_scr_rev = []

genre = []
language = []
director = []
producer = []
writer = []
rel_date_th = []
rel_date_stream = []
box = []
runtime = []
distributor = []
description = []

In [13]:
# name[0], name[2]

In [14]:
# name

In [15]:
# short_info

In [16]:
# selecionamos um pedaco da lista de links de filmes comencando pelo "start"
# links = links[start:]

def create_df(n):
    '''
    n: numero de filmes a coletar
    start: numero da lista do link do filme que vamos comecar a coletar
    '''  
    # neste dataset iremos armazenar as informacoes extraidas do Rotten Tomatoes
    df = pd.DataFrame()

    for j in range(len(links[:n])):
        
        # mostra quantos filmes ja varreu
        print(j, end="\r", flush=True)
        
        # entramos no site do filme
        driver.get(links[j])
        
        # se aparecer algun erro no navegador voltamos ao comeco do loop
        try:
            # informacoes da descricao do filme
            info_we = driver.find_elements(By.CLASS_NAME, "media-body")
            # informações mais breves da descricao
            short_info = info_we[0].text.split('\n')[2:]

            # informacoes principais do filme: nome, scores e quantidade de reviews
            info_main = driver.find_elements(By.CLASS_NAME, "thumbnail-scoreboard-wrap")
            info_main = info_main[0].text.split('\n')

            # dicionario para armazenar as infos do filme
            short_info_dic = dict()

            # acrescentamos tambem as informacoes principais
            short_info_dic['Nome'] = info_main[0]
            short_info_dic['TOMATOMETER'] = info_main[2]
            short_info_dic['TOMATOMETER reviews'] = info_main[4]
            short_info_dic['AUDIENCE SCORE'] = info_main[5]
            short_info_dic['AUDIENCE SCORE reviews'] = info_main[7]
        except:
            continue

        # informacoes breves do filme
        for i in range(len(short_info)):
            infos = short_info[i].split(': ')
            try:
                short_info_dic[infos[0]] = infos[1]
            except:
                short_info_dic[infos[0]] = np.nan

        # resumo e descricao do filme
        short_info_dic[info_we[0].text.split('\n')[0]] = info_we[0].text.split('\n')[1]

        # colocamos as informacoes nas listas
        try:
            nomes.append(short_info_dic['Nome'])
            tmdb_id.append(df0.loc[[j],['id']].values[0][0])
        except:
            nomes.append(np.nan)
        
        try:
            TOMATOMETER.append(short_info_dic['TOMATOMETER'])
        except:
            TOMATOMETER.append(np.nan)
        
        try:
            TOMATOMETER_rev.append(short_info_dic['TOMATOMETER reviews'])
        except:
            TOMATOMETER_rev.append(np.nan)
        
        try:
            aud_scr.append(short_info_dic['AUDIENCE SCORE'])
        except:
            aud_scr.append(np.nan)
        
        try:
            aud_scr_rev.append(short_info_dic['AUDIENCE SCORE reviews'])
        except:
            aud_scr_rev.append(np.nan)
        
        try:
            genre.append(short_info_dic['Genre'])
        except:
            genre.append(np.nan)
        
        try:
            language.append(short_info_dic['Original Language'])
        except:
            language.append(np.nan)
        
        try:
            director.append(short_info_dic['Director'])
        except:
            director.append(np.nan)
        
        try:
            producer.append(short_info_dic['Producer'])
        except:
            producer.append(np.nan)
        
        try:
            writer.append(short_info_dic['Writer'])
        except:
            writer.append(np.nan)
        
        try:
            rel_date_th.append(short_info_dic['Release Date (Theaters)'])
        except:
            rel_date_th.append(np.nan)
        
        try:
            rel_date_stream.append(short_info_dic['Release Date (Streaming)'])
        except:
            rel_date_stream.append(np.nan)
        
        try:
            box.append(short_info_dic['Box Office (Gross USA)'])
        except:
            box.append(np.nan)
        
        try:
            runtime.append(short_info_dic['Runtime'])
        except:
            runtime.append(np.nan)
        
        try:
            distributor.append(short_info_dic['Distributor'])
        except:
            distributor.append(np.nan)
            
        try:
            description.append(short_info_dic['MOVIE INFO'])
        except:
            description.append(np.nan)
            
    # Alimentamos o database

    df = pd.DataFrame() 

    df['tmdb_id'] = tmdb_id
    df['Movie Name'] = nomes
    df['Audience Score'] = aud_scr
    df['AUDIENCE SCORE reviews'] = aud_scr_rev
    df['Tomatometer Score'] = TOMATOMETER
    df['TOMATOMETER reviews'] = nomes
    df['Genre'] = genre
    df['Original Language'] = language
    df['Director'] = director
    df['Producer'] = producer
    df['Writer'] = writer
    df['Release Date (Theaters)'] = rel_date_th
    df['Release Date (Streaming)'] = rel_date_stream
    df['box office'] = box
    df['Runtime'] = runtime
    df['Distributor'] = distributor
    df['MOVIE INFO'] = description
        
    return df

In [17]:
# # conexao com o database do GCP
# con = psycopg2.connect(host='104.154.213.9' , database='turing_de',
# user = 'postgres', password='pwd')

In [18]:
# df

In [19]:
# %%timeit
# df = create_df(len(links))
df = create_df(4)

In [20]:
df

,tmdb_id,Movie Name,Audience Score,AUDIENCE SCORE reviews,Tomatometer Score,TOMATOMETER reviews,Genre,Original Language,Director,Producer,Writer,Release Date (Theaters),Release Date (Streaming),box office,Runtime,Distributor,MOVIE INFO
0,673,HARRY POTTER AND THE PRISONER OF AZKABAN,86%,"250,000+ Ratings",90%,HARRY POTTER AND THE PRISONER OF AZKABAN,"Kids & Family, Fantasy, Adventure",English,Alfonso Cuarón,"Chris Columbus, David Heyman, Mark Radcliffe","J.K. Rowling, Steve Kloves","May 31, 2004 Original","Nov 23, 2004",$249.4M,2h 21m,Warner Bros. Pictures,Harry Potter's (Daniel Radcliffe) third year a...
1,299537,WATCH TRAILER,550 Reviews,AUDIENCE SCORE,"PG-13 2019, Action/Adventure, 2h 4m",WATCH TRAILER,"Action, Adventure, Sci-Fi, Fantasy",English,"Anna Boden, Ryan Fleck",Kevin Feige,"Anna Boden, Ryan Fleck, Geneva Robertson-Dwore...","Mar 8, 2019 Wide","May 28, 2019",$426.8M,2h 4m,Walt Disney,Captain Marvel is an extraterrestrial Kree war...
2,335984,WATCH TRAILER,444 Reviews,AUDIENCE SCORE,"R 2017, Sci-fi/Mystery & thriller, 2h 44m",WATCH TRAILER,"Sci-Fi, Mystery & Thriller",English,Denis Villeneuve,"Andrew A. Kosove, Broderick Johnson, Bud Yorki...","Hampton Fancher, Michael Green","Oct 6, 2017 Wide","Jan 16, 2018",$91.5M,2h 44m,Warner Bros. Pictures,"Officer K (Ryan Gosling), a new blade runner f..."
3,1771,WATCH TRAILER,274 Reviews,AUDIENCE SCORE,"PG-13 2011, Action/Adventure, 2h 4m",WATCH TRAILER,"Action, Adventure, Fantasy",English,Joe Johnston,Kevin Feige,"Christopher Markus, Stephen McFeely","Jul 22, 2011 Wide","Oct 25, 2011",$176.6M,2h 4m,Paramount Pictures,It is 1941 and the world is in the throes of w...


In [21]:
# df.to_csv("data_output/Rotten_2021_4000.csv", index=False)

In [22]:
# # @timeit

# df.to_csv("data_output/Rotten_2021_4000.csv", index=False)

In [23]:
# pd.read_csv("data_output/Rotten_2021_4000.csv").head(50)

In [24]:
# filtrar por lingua e por ano